In [1]:
# !pip install harmonydata

In [2]:
import harmony

/home/thomas/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import pandas as pd

In [4]:
import numpy as np
from harmony import match_instruments_with_function, example_instruments
from openai import OpenAI

client = OpenAI()
model_name = "text-embedding-ada-002"
def convert_texts_to_vector(texts):
    vectors = client.embeddings.create(input = texts, model=model_name).data
    return np.asarray([vectors[i].embedding for i in range(len(vectors))])

In [5]:
df = pd.read_excel("gorilla-human-vs-machine-gorilla-vs-harmony.xlsx")

In [6]:
df

,id,human_similarity,Statement_1,Statement_2,cosine_from_harmony
0,16,60,"Trouble falling or staying asleep, or sleeping...",Felt nervous or anxious?,0.408625
1,20,20,Being “superalert” or watchful or on guard?,Felt hopeless?,-0.282368
2,24,40,I often ramble on too much when speaking.,Have had difficulty concentrating?,0.299198
3,28,70,Not being able to stop or control worrying?,Felt “on edge”?,-0.389295
4,32,25,"Thoughts that you would be better off dead, or...",Most of the time I feel a desire to be connect...,0.244937
...,...,...,...,...,...
2934,147,3,Do you sometimes get concerned that friends or...,Feeling jumpy or easily startled?,-0.147023
2935,152,0,Moving or speaking so slowly that other people...,Experienced reduced energy or fatigue?,0.229282
2936,157,0,When you look at a person or yourself in a mir...,Have had changes in appetite or sleep?,0.298841
2937,162,10,"Feeling down, depressed, or hopeless?",Generally I do not have many thoughts or emoti...,-0.443594


In [7]:
all_statements = sorted(set(list(df.Statement_1) + list(df.Statement_2)))

In [8]:
len(all_statements)

129

In [9]:
from harmony import create_instrument_from_list

In [10]:
instrument = create_instrument_from_list(all_statements)
instruments  =[instrument]

In [11]:
from harmony import match_instruments, match_instruments_with_function
all_questions, similarity, query_similarity, new_vectors_dict = match_instruments_with_function(instruments, None, convert_texts_to_vector)

In [12]:
df_similarity = pd.DataFrame(similarity)

In [13]:
df_similarity.columns = [q.question_text for q in all_questions]

In [14]:
df_similarity.index = [q.question_text for q in all_questions]

In [15]:
cosine_values = [None] * len(df)
for idx in range(len(df)):
    s1 = df["Statement_1"].iloc[idx]
    s2 = df["Statement_2"].iloc[idx]
    
    cos = df_similarity[s1].loc[s2]
    
    cosine_values[idx] = cos

In [16]:
df["cosine_openai"] = cosine_values

In [17]:
df

,id,human_similarity,Statement_1,Statement_2,cosine_from_harmony,cosine_openai
0,16,60,"Trouble falling or staying asleep, or sleeping...",Felt nervous or anxious?,0.408625,0.822943
1,20,20,Being “superalert” or watchful or on guard?,Felt hopeless?,-0.282368,-0.768003
2,24,40,I often ramble on too much when speaking.,Have had difficulty concentrating?,0.299198,0.784990
3,28,70,Not being able to stop or control worrying?,Felt “on edge”?,-0.389295,0.835432
4,32,25,"Thoughts that you would be better off dead, or...",Most of the time I feel a desire to be connect...,0.244937,0.761849
...,...,...,...,...,...,...
2934,147,3,Do you sometimes get concerned that friends or...,Feeling jumpy or easily startled?,-0.147023,0.806984
2935,152,0,Moving or speaking so slowly that other people...,Experienced reduced energy or fatigue?,0.229282,0.796771
2936,157,0,When you look at a person or yourself in a mir...,Have had changes in appetite or sleep?,0.298841,0.811275
2937,162,10,"Feeling down, depressed, or hopeless?",Generally I do not have many thoughts or emoti...,-0.443594,0.777102


In [18]:
df.to_csv("with_openai.csv", index=False)

In [19]:
from sklearn.linear_model import LinearRegression

In [20]:
model = LinearRegression()

In [21]:
model.fit(df[["cosine_openai"]], df["human_similarity"])

LinearRegression()

In [22]:
y_pred = model.predict(df[["cosine_openai"]])

In [23]:
from sklearn.metrics import mean_absolute_error

In [24]:
mean_absolute_error(df["human_similarity"], y_pred)

23.577036251687126